# Article Writing

This will be the most basic example you could create. 
You just define what type of input is accepted and use a language_task to query a AI model for a response.

```json
{
  "recipe": {
    "name": "Simple Article Generation",
    "user_inputs": [
      {
        "id": "topic",
        "label": "Topic of the article",
        "type": "string",
        "default": "The future of AI and its impact on society"
      }
    ],
    "nodes": [
      {
        "id": "generate_article",
        "type": "language_task",
        "model": {"provider": "openai", "model": "gpt-4o-mini", "temperature": 0.7},
        "prompt": "Write a well-structured article about {{topic}}. Make it engaging and informative."
      }
    ]
  }
}
```

In [1]:
from content_composer import parse_recipe, execute_workflow

# workflows can be defined in multiple formats, this example uses a json file
recipe = parse_recipe("./article.json")
recipe

2025-06-20 17:07:16.528 | INFO     | content_composer.registry.registry:initialize:26 - Function registry initialized with 0 functions


Recipe(name='Simple Article Generation', user_inputs=[UserInput(id='topic', label='Topic of the article', type='string', description=None, default='The future of AI and its impact on society', required=True, literal_values=None)], nodes=[Node(id='generate_article', type=<NodeType.LANGUAGE_TASK: 'language_task'>, description=None, prompt='Write a well-structured article about {{topic}}. Make it engaging and informative.', model=ModelConfig(provider='openai', model='gpt-4o-mini', temperature=0.7), function_identifier=None, input=None, output='generate_article', recipe_path=None, input_mapping=None, output_mapping=None, map_task_definition=None, map_over_key=None, map_on_error='halt', map_execution_mode='parallel')], edges=None, final_outputs=None)

Now, you just execute the workflow.

In [2]:
inputs = {"topic": "The future of AI and its impact on society"}
outputs = await execute_workflow(
    recipe,
    inputs
)
outputs

{'topic': 'The future of AI and its impact on society',
 'generate_article': "# The Future of AI and Its Impact on Society\n\nArtificial Intelligence (AI) has rapidly evolved from a niche technology into a pervasive force that is reshaping industries and daily life. As we look toward the future, it is essential to consider not only the advancements on the horizon but also the profound implications these developments will have on society as a whole. In this article, we will explore the future of AI, its potential benefits, challenges, and the ethical considerations that accompany its growth.\n\n## The Promise of AI: Transforming Industries\n\n### 1. Healthcare Revolution\n\nOne of the most promising areas for AI application is healthcare. With advancements in machine learning and data analytics, AI has the potential to revolutionize patient diagnosis and treatment. For instance, AI algorithms can analyze medical images with remarkable accuracy, enabling early detection of conditions suc

## Using Composition

In this example, we will reuse the article_writing recipe and extend it with a AI review to enhance its quality. This pipeline introduces some cool concepts, like composition and imports. 

Notice we are importing the model definitions from the common.yaml file. This is a common pattern in Content Composer recipes. But you can also create definitions in the same recipe file, which is the case with the prompt. We are using the @ symbol to reference the imported definitions. 

Also, notice the new type of node, recipe. This is a special node that allows you to reuse a full recipe as a part of another recipe. 

```yaml
imports:
  - "common.yaml"

# Recipe-specific definitions
definitions:
  expand_article_prompt: |
    Take the following draft article and expand it significantly. Add more details, examples, and flesh out the sections. 
    
    Draft Article:
    {{draft_article}}

recipe:
  name: Better Article Generation with Expansion
  user_inputs:
    - id: topic
      label: "The main topic of the article"
      type: text
      description: "The main topic of the article"
  nodes:
    - id: generate_initial_article
      type: recipe
      description: "Generate initial article using the article.yaml recipe"
      recipe_path: "article.json"
      input_mapping:
        topic: topic
      output_mapping:
        draft_article: generate_article
      output: draft_article
    - id: expand_article
      type: language_task
      model: "@reviewer"
      prompt: "@expand_article_prompt"
      input:
        draft_article: draft_article
      output: final_article
  edges:
    - "generate_initial_article to expand_article"
  final_outputs:
    - final_article
```


In [3]:
new_recipe = parse_recipe("better_article.yaml")
inputs = {"topic": "The future of AI and its impact on society"}
outputs = await execute_workflow(
    new_recipe,
    inputs
)
outputs

{'final_article': '# The Future of AI and Its Impact on Society\n\nArtificial Intelligence (AI) has transcended the realm of science fiction to become an integral part of our daily lives. Once a futuristic fantasy, AI now powers the smart assistants in our homes, like Amazon Alexa and Google Assistant, curates the content we see on social media platforms like Instagram and TikTok, and even influences critical decisions in industries ranging from healthcare to finance. As we stand on the cusp of a technological revolution, it is vital to delve deeper into the future of AI, exploring not only its potential to reshape society but also the challenges and responsibilities that come with such profound change.\n\nThis article aims to provide a comprehensive look at AI\'s trajectory by examining its historical evolution, transformative applications across various sectors, ethical dilemmas, and the importance of robust governance. By understanding these facets, we can better prepare for a futur